In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime as dt

In [2]:
SUMMER_2018_DATE = dt.date(2018, 6, 1)

In [3]:
base_url = 'https://www.transfermarkt.com/a/marktwertverlauf/spieler/' # "a" is a placeholder, can be anything, technically is player name but doesn't matter

In [4]:
headers = {'User-Agent': 
       'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [5]:
transfermarkt_ids = [148153, 82873, 209212] # sample player IDs for Gross, March and Dunk

In [6]:
columns = ['player_name', 'transfermarkt_player_id', 'date', 'market_value_in_gbp']

In [7]:
def get_market_value_history_of_player(transfermarkt_player_id, maximum_date=None, as_pandas=False):
    tree = requests.get(base_url + str(transfermarkt_player_id), headers=headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
    
    script = soup.find('script', text=re.compile('Highcharts.Chart')).text
    player_name = soup.find('title').text.partition(' - ')[0]
    
    chart_data = script.split("'data':[")[1]
    ending_string = '}}]'
    chart_data = chart_data[:chart_data.find(ending_string) + 1]

    delim = '},'
    market_values = [eval(e + delim[0]) for e in chart_data.split(delim) if e[:5] == "{'y':"]

    market_vals_clean = [(player_name, transfermarkt_player_id, dt.date.fromtimestamp(int(str(data_point['x'])[:-3])), data_point['y']) for data_point in market_values]

    if maximum_date:
        market_vals_clean = [[record for record in market_vals_clean if record[2] < maximum_date][-1]]

    if as_pandas:
        market_vals_clean = pd.DataFrame(market_vals_clean, columns=columns)

    return market_vals_clean

In [8]:
all_player_market_values = []

for id in transfermarkt_ids:
    all_player_market_values.extend(get_market_value_history_of_player(transfermarkt_player_id=id, maximum_date=SUMMER_2018_DATE, as_pandas=False))

In [9]:
df_all_player_market_values = pd.DataFrame(all_player_market_values, columns=columns)
df_all_player_market_values

,player_name,transfermarkt_player_id,date,market_value_in_gbp
0,Lewis Dunk,148153,2018-05-28,10000000
1,Pascal Groß,82873,2018-05-28,10000000
2,Solly March,209212,2018-05-28,5000000
